# Monet GAN Project

Intro here ...

In [1]:
import tensorflow as tf
import os

2025-11-01 11:47:14.416537: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 11:47:14.458596: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-01 11:47:15.974856: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Data Import

- Data was imported from the "I’m Something of a Painter Myself" Kaggle competition.
- 2 folders provided from Kaggle: monet_jpg and photo_jpg. These will be converted to Tensorflow record files so it's easier to do work with
- 

Reference:  
Amy Jang, Ana Sofia Uzsoy, and Phil Culliton. I’m Something of a Painter Myself. https://kaggle.com/competitions/gan-getting-started, 2020. Kaggle.

#### Convert Photos to .tfrec format
- Why I chose to use .tfrec (what are the advantages vs processing .png?)

In [ ]:

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def image_to_tfexample(image_string, label):
    feature = {
        "image": _bytes_feature(image_string),
        "label": _bytes_feature(label.encode('utf-8'))
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

def convert_images_to_tfrecords(input_dir, output_dir, label):
    os.makedirs(output_dir, exist_ok=True)
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".jpg")]

    for i, filename in enumerate(image_files, 1):
        image_path = os.path.join(input_dir, filename)
        tfrec_name = os.path.splitext(filename)[0] + ".tfrec"
        tfrec_path = os.path.join(output_dir, tfrec_name)

        with open(image_path, "rb") as f:
            image_bytes = f.read()

        example = image_to_tfexample(image_bytes, label)
        with tf.io.TFRecordWriter(tfrec_path) as writer:
            writer.write(example.SerializeToString())

        if i % 100 == 0 or i == len(image_files):
            print(f"Processed {i}/{len(image_files)} images into {output_dir}")

# Define folders
photo_jpg_dir = "photo_jpg"
monet_jpg_dir = "monet_jpg"
photo_tfrec_dir = "photo_tfrec"
monet_tfrec_dir = "monet_tfrec"

# Run conversions
convert_images_to_tfrecords(photo_jpg_dir, photo_tfrec_dir, "photo")
convert_images_to_tfrecords(monet_jpg_dir, monet_tfrec_dir, "monet")

print("All .jpg files have been converted to individual .tfrec files.")

Processed 100/7038 images into photo_tfrec
Processed 200/7038 images into photo_tfrec
Processed 300/7038 images into photo_tfrec
Processed 400/7038 images into photo_tfrec
Processed 500/7038 images into photo_tfrec
Processed 600/7038 images into photo_tfrec
Processed 700/7038 images into photo_tfrec
Processed 800/7038 images into photo_tfrec
Processed 900/7038 images into photo_tfrec
Processed 1000/7038 images into photo_tfrec
Processed 1100/7038 images into photo_tfrec
Processed 1200/7038 images into photo_tfrec
Processed 1300/7038 images into photo_tfrec
Processed 1400/7038 images into photo_tfrec
Processed 1500/7038 images into photo_tfrec
Processed 1600/7038 images into photo_tfrec
Processed 1700/7038 images into photo_tfrec
Processed 1800/7038 images into photo_tfrec
Processed 1900/7038 images into photo_tfrec
Processed 2000/7038 images into photo_tfrec
Processed 2100/7038 images into photo_tfrec
Processed 2200/7038 images into photo_tfrec
Processed 2300/7038 images into photo_tfr